In [14]:
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import swifter
from fastai.text import *
path = Path('/home/jupyter/tutorials/Thathasthu')
import gensim
from spellchecker import SpellChecker
spell = SpellChecker()

In [112]:
!pip install boto

In [24]:

train = pd.read_csv('./tathasthu/ttrain.csv')
manufacturers = pd.read_csv("./tathasthu/manufacturer_list.csv")


In [78]:
def clean_column(column, delimitter_list):
    column = column.str.lower()
    for delimitter in delimitter_list: 
        column = column.str.replace(delimitter," ")
    column = column.str.replace("  "," ")
    column = column.str.replace("  "," ")
    column = column.str.replace("  "," ")
    column = column.str.replace("  "," ")
    column = column.str.replace("  "," ")
    column = column.str.replace("  "," ")
    return column

def get_brand_name(article_name, choices):
    maxScore = 0
    maxChoice = ""
    for choice in choices:
        score = fuzz.token_set_ratio(" " + article_name + " ", " " + choice + " ")
        if score > maxScore or (score == maxScore and len(choice) > len(maxChoice)):
            maxScore = score
            maxChoice = choice
    return maxChoice, maxScore

def get_strict(article_name, choices):
    matches = []
    for choice in choices:
        choice1 = choice + " "
        choice2 = " " + choice + " "
        choice3 = " " + choice
        if article_name.startswith(choice1) or article_name.endswith(choice3) or choice2 in article_name:
            matches.append(choice)
    return matches

def remove_string(large, queries):
    for query in queries.split(" "):
        large = large.replace(query, "")
        large = large.replace(query+"s", "")
    large = large.replace("  ", " ")
    large = large.replace("  ", " ")
    large = large.replace("  ", " ")
    return large

def get_type(filtered_string):
    type_column = set()
    for word in filtered_string.split(' '):
        for seed in seed_choices:
            try: 
                similarity_score = model.similarity(seed, word)
            except KeyError:
                similarity_score = 0.0
            if similarity_score > 0.35:
                type_column.add(word)
    #         print(seed + " " + word + " " + str(similarity_score))
    s = list(type_column)
    string_array = ' '.join(str(e) for e in s)
    return string_array

def get_words_from_sentence(sentence):
    words = sentence.split(" ")
    returns = []
    misspelled = spell.unknown(words)
    for word in words:
        if len(word) > 2 and word not in misspelled and word not in stopwords:
            returns.append(word)
    return returns

seed_choices = ['oranges', 'apples', 'avocado', 'onion', 'tomato', 'pants', 'shirt', 'pink', 'blue']
seed_colors = ['red', 'blue', 'green']
seed_flavors = ['onion', 'tomato', 'apples', 'avocado']
seed_type = ['shirt', 'pant', 'soap', 'cosmetics']

def get_type_specific(filtered_string):
    type_column = set()
    for word in filtered_string.split(' '):
        for seed in seed_colors:
            try:
                similarity_score = model.similarity(seed, word)
            except KeyError:
                similarity_score = 0.0
            if similarity_score > 0.35:
                 type_column.add('color: ' + word)
       
        for seed in seed_flavors:
            try:
                similarity_score = model.similarity(seed, word)
            except KeyError:
                similarity_score = 0.0
            if similarity_score > 0.35:
                type_column.add('flavor: ' + word)

        for seed in seed_type:
            try:
                similarity_score = model.similarity(seed, word)
            except KeyError:
                similarity_score = 0.0
            if similarity_score > 0.35:
                type_column.add('type: ' + word)
    
    s = list(type_column)
    string_array = ' '.join(str(e) for e in s)
    return string_array

In [30]:
train['manufacturer_description'] = clean_column(train['manufacturer_description'], [])

In [31]:
train["company_name"] = train.manufacturer_description.str.extract(r"company name - ([a-zA-Z-09\. \&]+)")

In [32]:
train["article_name"] = clean_column(train['article_name'], ["'", "(", ")", "/"])

In [33]:
train['quantity'] = train['article_name'].str.extract(r'(\d.{0,4}(ml|g|kg|l|s|p))')[0]

In [34]:
manufacturers["brand_name"] = clean_column(manufacturers['brand_name'], ["'", "(", ")", "/"])

In [35]:
train["company_name"] = train["company_name"].fillna("")

In [36]:
train["article_name"] = train["article_name"].fillna("")

In [37]:
train['strict_transformed'] = train['article_name'].apply(lambda x: get_strict(x, manufacturers['brand_name']))

In [38]:
train['strict_company'] = train['company_name'].apply(lambda x: get_strict(x, manufacturers['brand_name']))

In [39]:
train = train[train["article_name"] != ""]

In [40]:
train

,article_name,manufacturer_description,Unnamed: 2,Unnamed: 3,Unnamed: 4,company_name,quantity,strict_transformed,strict_company
0,chicken burger patty 250g,company name - venkys india ltd<br />membershi...,NaN,NaN,NaN,venkys india ltd,250g,[],[venkys]
1,britannia daily family long bread,company name - britannia industries ltd<br />m...,NaN,NaN,NaN,britannia industries ltd,NaN,[britannia],[britannia]
2,pooof potato kettle chips zesty masala pp 30g,company name - future consumer enterprise limi...,NaN,NaN,NaN,future consumer enterprise limited,30g,[],[]
3,fevibond tube 20 ml,membership discount - applicable<br />identifi...,NaN,NaN,NaN,,20 ml,[fevibond],[]
4,kitkat dessert delight truffle 50g,company name - nestle india ltd<br />membershi...,NaN,NaN,NaN,nestle india ltd,50g,[],[nestle]
5,blue bird spaghetti pasta 200g,company name - blue bird foods india pvt ltd<b...,NaN,NaN,NaN,blue bird foods india pvt ltd,200g,[blue bird],[blue bird]
6,nirapara avalose podi 500gm,company name - k k r food products<br />member...,NaN,NaN,NaN,k k r food products,500g,[nirapara],[]
7,nivea roll on fresh natural 50ml,membership discount - applicable<br />identifi...,NaN,NaN,NaN,,50ml,[nivea],[]
8,fiama dw soap exotic dream bx 115g,company name - itc india ltd<br />membership d...,NaN,NaN,NaN,itc india ltd,115g,[],[]
9,sangis ktcn peri peri mayonnaise bt 200g,company name - future consumer ltd<br />member...,NaN,NaN,NaN,future consumer ltd,200g,[],[]


In [41]:
train['fuzzy_transformed'] = train['article_name'].swifter.apply(lambda x: get_brand_name(x, manufacturers['brand_name']))


Pandas Apply:   9%|▉         | 809/8621 [00:12<01:57, 66.74it/s]


Pandas Apply:  19%|█▉        | 1670/8621 [00:25<01:43, 67.07it/s]


Pandas Apply:  29%|██▉       | 2517/8621 [00:37<01:30, 67.20it/s]


Pandas Apply:  39%|███▉      | 3353/8621 [00:50<01:19, 66.01it/s]


Pandas Apply:  49%|████▊     | 4187/8621 [01:02<01:08, 64.55it/s]


...

Pandas Apply:  67%|██████▋   | 5792/8621 [01:28<00:39, 71.38it/s]


Pandas Apply:  77%|███████▋  | 6640/8621 [01:40<00:29, 67.16it/s]


Pandas Apply:  87%|████████▋ | 7474/8621 [01:53<00:17, 66.95it/s]


Pandas Apply:  96%|█████████▋| 8313/8621 [02:06<00:04, 66.33it/s]


Pandas Apply: 100%|██████████| 8621/8621 [02:10<00:00, 65.91it/s]/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [42]:
train['fuzzy_company'] = train['company_name'].swifter.apply(lambda x: get_brand_name(x, manufacturers['brand_name']))


Pandas Apply:  11%|█▏        | 981/8621 [00:13<01:48, 70.63it/s]


Pandas Apply:  23%|██▎       | 1974/8621 [00:25<01:29, 74.36it/s]


Pandas Apply:  34%|███▍      | 2955/8621 [00:38<01:15, 75.38it/s]


Pandas Apply:  45%|████▌     | 3920/8621 [00:51<01:02, 75.78it/s]


Pandas Apply:  56%|█████▋    | 4851/8621 [01:04<00:57, 65.01it/s]


Pandas Apply:  68%|██████▊   | 5862/8621 [01:17<00:38, 72.47it/s]


Pandas Apply:  79%|███████▉  | 6845/8621 [01:30<00:22, 77.82it/s]


Pandas Apply:  91%|█████████ | 7850/8621 [01:43<00:09, 78.80it/s]


Pandas Apply: 100%|██████████| 8621/8621 [01:53<00:00, 75.73it/s]/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [43]:
train = train.filter(['article_name','company_name', 'quantity', 'strict_transformed', 'strict_company', 'fuzzy_transformed', 'fuzzy_company']).join(train.fuzzy_transformed.apply(lambda loc: pd.Series(loc, index=['fuzzy_brand', 'brand_score']))).join(train.fuzzy_company.apply(lambda loc: pd.Series(loc, index=['f_company', 'company_score'])))



In [44]:
del train['fuzzy_transformed']
del train['fuzzy_company']

In [45]:
train["final_brand"] = train.swifter.apply(lambda x: x.fuzzy_brand if x.brand_score > 50 else x.strict_company[0] if len(x.strict_company) > 0 else None, axis=1)



Pandas Apply: 100%|██████████| 8621/8621 [00:00<00:00, 34961.52it/s]

In [46]:
train = train.filter(["article_name", "quantity", "final_brand"])

In [47]:
train["filtered"] = train.apply(lambda x: x.article_name if x.final_brand is None else remove_string(x.article_name, x.final_brand), axis=1)


In [48]:
train["filtered"] = train.apply(lambda x: remove_string(x.filtered, str(x.quantity)), axis=1)

In [49]:
train.to_csv("train_quantity.csv", index=False)

In [50]:
learn = load_learner(path)

In [51]:
train["final_brand"] = train.swifter.apply(lambda x: "" if x.final_brand == None else x.final_brand, axis=1)


Pandas Apply: 100%|██████████| 8621/8621 [00:00<00:00, 43905.89it/s]

In [52]:
train['function_dl'] = train.swifter.apply(lambda x: str(learn.predict(x['final_brand']+' '+x['filtered'])[0]), axis=1)


Pandas Apply:   4%|▎         | 307/8621 [00:15<07:30, 18.45it/s]


Pandas Apply:   7%|▋         | 619/8621 [00:30<06:21, 20.96it/s]


Pandas Apply:  11%|█▏        | 979/8621 [00:47<05:50, 21.79it/s]


Pandas Apply:  15%|█▌        | 1333/8621 [01:03<05:51, 20.76it/s]


Pandas Apply:  20%|█▉        | 1687/8621 [01:20<05:35, 20.64it/s]


...

Pandas Apply:  88%|████████▊ | 7567/8621 [06:07<00:47, 22.28it/s]


Pandas Apply:  92%|█████████▏| 7921/8621 [06:24<00:33, 20.63it/s]


Pandas Apply:  96%|█████████▌| 8275/8621 [06:41<00:16, 21.43it/s]


Pandas Apply: 100%|█████████▉| 8592/8621 [06:56<00:01, 18.24it/s]


Pandas Apply: 100%|██████████| 8621/8621 [06:58<00:00, 20.61it/s]

In [53]:
function_df = pd.read_csv("./classes")

In [54]:
function_filtered = pd.merge(train,function_df, how='left', left_on = 'function_dl', right_on = 'class1')

In [55]:
function_filtered['class1'] = clean_column(function_filtered['class1'], ["'", "(", ")", "/", "&", ","])

In [56]:
function_filtered['class2'] = clean_column(function_filtered['class2'], ["'", "(", ")", "/", "&", ","])

In [57]:
function_filtered['appended_class'] = function_filtered.apply(lambda x: x.class1 + " " + x.class2, axis=1)

In [58]:
function_filtered["cleaned"] = function_filtered.apply(lambda x: remove_string(x.filtered, x.appended_class), axis=1)

In [60]:
final1_data = function_filtered.filter(['article_name', 'cleaned', 'quantity', 'final_brand', 'class1', 'class2'])

In [61]:
final1_data.to_csv('./final1_data.csv', index=False)

In [62]:
final1_data = pd.read_csv('./final1_data.csv')

In [63]:
model = gensim.models.KeyedVectors.load_word2vec_format('./google.bin', binary=True)  



In [64]:
seed_choices = ['oranges', 'apples', 'avocado', 'onion', 'tomato', 'pants', 'shirt', 'pink', 'blue']
get_type("kitkat dessert delight truffle")

'truffle dessert'

In [65]:
final1_data["cleaned"] = final1_data.cleaned.fillna("")

In [79]:
final1_data['type'] = final1_data.swifter.apply(lambda x:get_type_specific(x.cleaned), axis=1)


Pandas Apply: 100%|██████████| 8621/8621 [00:08<00:00, 1024.17it/s]

In [81]:
final1_data

,article_name,cleaned,quantity,final_brand,class1,class2,type,cleaned_more,get_cleaned
0,chicken burger patty 250g,chicken burger patty,250g,venkys,ready to eat,branded food,flavor: chicken flavor: patty flavor: burger,,[]
1,britannia daily family long bread,daily family long bread,NaN,britannia,biscuits cookies crackers,branded food,flavor: bread,daily family long,"[daily, family, long]"
2,pooof potato kettle chips zesty masala pp 30g,pooof potato kettle zesty masala pp,30g,pooof!,chips namkeens snacks,branded food,flavor: potato flavor: zesty,pooof kettle masala pp,"[kettle, masala]"
3,fevibond tube 20 ml,tube,20 ml,fevibond,otc,beauty personal care,,tube,[tube]
4,kitkat dessert delight truffle 50g,kitkat dessert delight truffle,50g,nestle,chocolates cakes confectioneries,branded food,flavor: truffle flavor: dessert,kitkat delight,[delight]
5,blue bird spaghetti pasta 200g,spaghetti pasta,200g,blue bird,noodles pastas vermicelli,branded food,flavor: pasta,spaghetti,[spaghetti]
6,nirapara avalose podi 500gm,avalose podi m,500g,nirapara,toys,toys stationery,,avaloe podi m,[]
7,nivea roll on fresh natural 50ml,roll on fresh natural,50ml,nivea,deos perfumes,beauty personal care,,roll on freh natural,"[roll, natural]"
8,fiama dw soap exotic dream bx 115g,dw soap exotic dream bx,115g,fiama di wills,skin care,beauty personal care,type: soap,dw oap exotic dream bx,"[oap, exotic, dream]"
9,sangis ktcn peri peri mayonnaise bt 200g,sangis ktcn peri peri mayonnaise bt,200g,NaN,jams spreads honey,branded food,type: mayonnaise flavor: mayonnaise,sangis ktcn peri peri bt,"[peri, peri]"


In [82]:
final1_data['cleaned_more'] = final1_data.swifter.apply(lambda x: remove_string(x.cleaned, x.type), axis=1)


Pandas Apply: 100%|██████████| 8621/8621 [00:00<00:00, 35011.01it/s]

In [84]:
final1_data

,article_name,cleaned,quantity,final_brand,class1,class2,type,cleaned_more,get_cleaned
0,chicken burger patty 250g,chicken burger patty,250g,venkys,ready to eat,branded food,flavor: chicken flavor: patty flavor: burger,,[]
1,britannia daily family long bread,daily family long bread,NaN,britannia,biscuits cookies crackers,branded food,flavor: bread,daily family long,"[daily, family, long]"
2,pooof potato kettle chips zesty masala pp 30g,pooof potato kettle zesty masala pp,30g,pooof!,chips namkeens snacks,branded food,flavor: potato flavor: zesty,pooof kettle masala pp,"[kettle, masala]"
3,fevibond tube 20 ml,tube,20 ml,fevibond,otc,beauty personal care,,tube,[tube]
4,kitkat dessert delight truffle 50g,kitkat dessert delight truffle,50g,nestle,chocolates cakes confectioneries,branded food,flavor: truffle flavor: dessert,kitkat delight,[delight]
5,blue bird spaghetti pasta 200g,spaghetti pasta,200g,blue bird,noodles pastas vermicelli,branded food,flavor: pasta,spaghetti,[spaghetti]
6,nirapara avalose podi 500gm,avalose podi m,500g,nirapara,toys,toys stationery,,avaloe podi m,[]
7,nivea roll on fresh natural 50ml,roll on fresh natural,50ml,nivea,deos perfumes,beauty personal care,,roll on freh natural,"[roll, natural]"
8,fiama dw soap exotic dream bx 115g,dw soap exotic dream bx,115g,fiama di wills,skin care,beauty personal care,type: soap,dw exotic dream bx,"[oap, exotic, dream]"
9,sangis ktcn peri peri mayonnaise bt 200g,sangis ktcn peri peri mayonnaise bt,200g,NaN,jams spreads honey,branded food,type: mayonnaise flavor: mayonnaise,sangis ktcn peri peri bt,"[peri, peri]"


In [85]:
stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

In [86]:
final1_data["get_cleaned"] = final1_data.swifter.apply(lambda x: get_words_from_sentence(x.cleaned_more), axis=1)


Pandas Apply: 100%|██████████| 8621/8621 [00:00<00:00, 33355.04it/s]

In [87]:
final1_data

,article_name,cleaned,quantity,final_brand,class1,class2,type,cleaned_more,get_cleaned
0,chicken burger patty 250g,chicken burger patty,250g,venkys,ready to eat,branded food,flavor: chicken flavor: patty flavor: burger,,[]
1,britannia daily family long bread,daily family long bread,NaN,britannia,biscuits cookies crackers,branded food,flavor: bread,daily family long,"[daily, family, long]"
2,pooof potato kettle chips zesty masala pp 30g,pooof potato kettle zesty masala pp,30g,pooof!,chips namkeens snacks,branded food,flavor: potato flavor: zesty,pooof kettle masala pp,"[kettle, masala]"
3,fevibond tube 20 ml,tube,20 ml,fevibond,otc,beauty personal care,,tube,[tube]
4,kitkat dessert delight truffle 50g,kitkat dessert delight truffle,50g,nestle,chocolates cakes confectioneries,branded food,flavor: truffle flavor: dessert,kitkat delight,[delight]
5,blue bird spaghetti pasta 200g,spaghetti pasta,200g,blue bird,noodles pastas vermicelli,branded food,flavor: pasta,spaghetti,[spaghetti]
6,nirapara avalose podi 500gm,avalose podi m,500g,nirapara,toys,toys stationery,,avaloe podi m,[]
7,nivea roll on fresh natural 50ml,roll on fresh natural,50ml,nivea,deos perfumes,beauty personal care,,roll on freh natural,"[roll, natural]"
8,fiama dw soap exotic dream bx 115g,dw soap exotic dream bx,115g,fiama di wills,skin care,beauty personal care,type: soap,dw exotic dream bx,"[exotic, dream]"
9,sangis ktcn peri peri mayonnaise bt 200g,sangis ktcn peri peri mayonnaise bt,200g,NaN,jams spreads honey,branded food,type: mayonnaise flavor: mayonnaise,sangis ktcn peri peri bt,"[peri, peri]"


In [88]:
final1_data.to_csv("./final_datav1.1.1.csv", index=False)